First we must import numpy and SINDy

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pysindy import SINDy
from pysindy.feature_library import *
from pysindy.differentiation import *
# from pysindy.optimizers import *
from scipy.integrate import odeint, solve_ivp

file = '../ReducedTorqueData.csv'
data = np.loadtxt(file, delimiter=',', skiprows=1, max_rows=100000)
pd.read_csv(file, nrows=3)

,timeInS,i_aInA,i_bInA,i_cInA,u_aInV,u_bInV,u_cInV,epsilon_elInRad
0,0.000000,1.765625,3.453125,0.937500,-0.322917,-0.677083,1.000000,-2.403804
1,0.000001,1.578125,3.421875,0.859375,-0.177083,-0.750000,0.927083,-2.403765
2,0.000002,1.578125,3.562500,0.750000,-0.114583,-0.854167,0.968750,-2.403726


In [3]:
# Given Parameters
n_me = 16.67    # Hz
U_DC = 294      # Volts DC
Rs = 18e-3      # Ohms
Ld = 370e-6     # Henry
Lq = 1200e-6    # Henry
psi_p = 66e-3   # Vs
i_dqmax = 240   # Amps
p = 3           # pole pair number

In [4]:
# Calculate Torque from variables
t = data[:, 0]
i_abc = data[:, 1:4]
u_abc = data[:, 4:7]
e = data[:, 7]

i_dq = np.zeros((len(data), 2))
u_dq = np.zeros((len(data), 2))
for i in range(len(data)):
    trig = np.array([[np.cos(e[i]), -np.cos(e[i] + np.pi/3), -np.cos(e[i] - np.pi/3)],
                    [-np.sin(e[i]), np.sin(e[i] + np.pi/3), np.sin(e[i] - np.pi/3)]])
    i_dq[i] = np.matmul(trig, i_abc[i])
    u_dq[i] = np.matmul(trig, u_abc[i])

In [5]:
psi_d = Ld*i_dq[:, 0] + psi_p
psi_q = Lq*i_dq[:, 1]
T = 1.5*p*(psi_d*i_dq[:, 1] - psi_q*i_dq[:, 0])

X = np.column_stack((T, i_dq, u_dq, np.sin(e), np.cos(e)))

In [17]:
# Instantiate model
model = SINDy(
              feature_library=PolynomialLibrary(degree=3),
              # feature_library=FourierLibrary(n_frequencies=4),
              # feature_library=WeakPDELibrary(),
              differentiation_method=SmoothedFiniteDifference(),
              # differentiation_method=SINDyDerivative(kind="kalman", alpha = 0.05),
              feature_names=['T','i_d','i_q','u_d','u_q','sin(e)','cos(e)']
              # feature_names=['T','i_d','i_q','u_d','u_q']
            )

model.fit(X, t=t)
model.print()
model.coefficients()
model.score(X, t=t[1]-t[0])

(T)' = 647755206931.213 1 + 270.310 T + 4589.295 i_d + 1418.386 i_q + -2217.247 u_d + -2295.267 u_q + -9082.247 sin(e) + 20002.100 cos(e) + -16690.387 T^2 + 1819.455 T i_d + 3777.872 T i_q + -377.918 T u_d + -271.678 T u_q + 4151.222 T sin(e) + 3995.178 T cos(e) + -24773.851 i_d^2 + 5685.662 i_d i_q + 3007.243 i_d u_d + -1444.026 i_d u_q + 22319.906 i_d sin(e) + -6488.450 i_d cos(e) + 967.350 i_q^2 + -1161.745 i_q u_d + -923.981 i_q u_q + -2594.374 i_q sin(e) + 9286.894 i_q cos(e) + 28.437 u_d^2 + 63.307 u_d u_q + -172.805 u_d sin(e) + -3538.893 u_d cos(e) + 40.678 u_q^2 + -641.127 u_q sin(e) + -1.175 u_q cos(e) + -647755211371.893 sin(e)^2 + 23429.648 sin(e) cos(e) + -647755210772.493 cos(e)^2 + 485836423.268 T^3 + -5444912.134 T^2 i_d + -474175792.722 T^2 i_q + 703986.805 T^2 u_d + 673281.458 T^2 u_q + -161416933.006 T^2 sin(e) + -96540528.380 T^2 cos(e) + 60175.876 T i_d^2 + 4767289.802 T i_d i_q + -26970.792 T i_d u_d + -525.054 T i_d u_q + 4477857.402 T i_d sin(e) + 1106021.557 T 

0.3140004801844823

In [18]:
tspan = [t[0], t[-1]]
x0 = X[0]
# x_SINDy = solve_ivp(model, tspan, x0, t_eval=t)

In [8]:
# sim = model.simulate(x0, t=t)       # sim should match X if there was no error
# print(sim)

In [19]:
# TorqueSINDy = sim[:,0]

In [ ]:
# # Define weak form ODE library
# # defaults to derivative_order = 0 if not specified,
# # and if spatial_grid is not specified, defaults to None,
# # which allows weak form ODEs.
# library_functions = [lambda x: x, lambda x: x * x, lambda x, y: x * y]
# library_function_names = [lambda x: x, lambda x: x + x, lambda x, y: x + y]
# ode_lib = ps.WeakPDELibrary(
#     library_functions=library_functions,
#     function_names=library_function_names,
#     spatiotemporal_grid=t_train,
#     is_uniform=True,
#     K=100,
)

In [ ]:
# plt.figure(figsize=(12,5))
# # plt.plot(t, T)
# # plt.plot(t, X)
# plt.plot(sim[:, 0], sim[:, 1])
# plt.xlabel('t')
# plt.ylabel('Torque')
# plt.legend()
# plt.show()